In [102]:
import math
# Erlang-B formula  
def erlang_b(a, k):
    '''
    a: in Erlangs, Traffic intensity
    k: number of line (channels)
    '''
    numerator = (a ** k) / math.factorial(k)
    denominator = sum((a ** i) / math.factorial(i) for i in range(k + 1))
    return numerator / denominator

In [103]:
call_arrival_rates = [2, 4, 9, 11, 10]  # calls per minute
mean_call_duration = 2  # minutes
total_channels = 48

In [132]:
# return the channel range for each cell
neighbors = {
    1: [2, 3],
    2: [1, 3],
    3: [1, 2, 4, 5],
    4: [3, 5],
    5: [3, 4]
}
def channel_range(total_channels):
    min_r = call_arrival_rates.copy()
    channel_range = {}
    for i in range(5):
        channel_range[i+1] = [min_r[i], total_channels - sum([min_r[n-1] for n in neighbors[i+1]])]
    return channel_range

In [133]:
# Function to calculate overall blocking probability  
def overall_blocking_probability(channel_allocation):
    total_blocking_prob = 0
    traffic_intensities = call_arrival_rates * mean_call_duration
    call_probabilities = [call_rate / sum(call_arrival_rates) for call_rate in call_arrival_rates]
    
    for i in range(len(channel_allocation)):
        k_i = channel_allocation[i]
        a_i = traffic_intensities[i]
        if k_i > 0:
            P_i = erlang_b(a_i, k_i)
        else:
            P_i = 1  # If no channels are allocated, all calls are blocked
        total_blocking_prob += call_probabilities[i] * P_i
    return total_blocking_prob

In [130]:
# Function to determine the distribution of the 48 channels over the five cells that minimizes the overall blocking probability
def optimize_by_fixed_channels(channel_distribution, per_cell_range, total_channels, set_target=False, target_prob = 0.01):
    best_prob = overall_blocking_probability(channel_distribution)
    # iterate based on the proportional contribution to the overall result
    for k in range(per_cell_range[1][0],per_cell_range[1][1]):
        for m in range(per_cell_range[2][0],per_cell_range[2][1]):
            for i in range(per_cell_range[3][0],per_cell_range[3][1]):
                for n in range(per_cell_range[5][0],per_cell_range[5][1]):
                    for j in range(per_cell_range[4][0],per_cell_range[4][1]):
                        new_distribution = [k, m, i, j, n]
                        if (sum(new_distribution[:3])> total_channels or sum(new_distribution[2:]) > total_channels):
                            break
                        new_prob = overall_blocking_probability(new_distribution)  # calculate overall blocking probability
                        if new_prob < best_prob:
                            best_distribution = new_distribution
                            best_prob = new_prob
                        if set_target and best_prob < target_prob:
                            return best_distribution, best_prob
    return best_distribution, best_prob


In [134]:
# Question 14 computation
per_cell_range = channel_range(total_channels=48)
channel_distribution = call_arrival_rates.copy()
optimal_distribution, optimal_prob = optimize_by_fixed_channels(channel_distribution, per_cell_range, total_channels=48, set_target = False)
print(f"Optimal channel distribution: {optimal_distribution}")
print(f"Minimum overall blocking probability: {optimal_prob:.4f}")

Optimal channel distribution: [14, 19, 15, 17, 16]
Minimum overall blocking probability: 0.0187


In [141]:
# Question 15 computation
target_prob = 0.01
total_channels = 48
for i in range(10):
    total_channels = total_channels + i +1
    per_cell_range = channel_range(total_channels)
    channel_distribution = call_arrival_rates.copy()
    optimal_distribution, optimal_prob = optimize_by_fixed_channels(channel_distribution, per_cell_range, total_channels, target_prob)
    if optimal_prob < target_prob:
        break


In [142]:
print(f"Optimal channel distribution: {optimal_distribution}")
print(f"Minimum overall blocking probability: {optimal_prob:.5f}")

Optimal channel distribution: [5, 9, 16, 20, 18]
Minimum overall blocking probability: 0.00969


In [143]:
extra_channel = max(sum(optimal_distribution[:3]),sum(optimal_distribution[2:])) - 48
print(extra_channel)

6
